# Graph RAG using Amazon Bedrock Knowledge Bases
In this module, you'll learn how to improve Generative AI applications using Retrieval-Augmented Generation (RAG) combined with Amazon Bedrock Knowledge Bases, enabling more accurate, contextual, and explainable responses combined with graph data.

The following steps will guide you through the process of using Amazon Bedrock Knowledge Bases with GraphRAG to enhance Generative AI applications.
This module contains:
1. [Overview](#Overview)
2. [Pre-requisites](#Pre-requisites)
3. [Setup](#Setup)
4. [Create KB with Amazon Neptune as vector](#Create-KB-with-Amazon-Neptune-vector)
5. [Test the Knowledge Graph](#Test-the-Knowledge-Graph)


## Overview
Amazon Bedrock Knowledge Bases offers a fully managed GraphRAG feature with Amazon Neptune. This functionality uses Retrieval Augmented Generation (RAG) techniques combined with graphs to enhance generative AI applications so that end users can get more accurate and comprehensive responses.

GraphRAG automatically identifies and uses relationships between related entities and structural elements (such as section titles) across documents that are ingested into Amazon Bedrock Knowledge Bases. This means that generative AI applications can deliver more relevant responses in cases where connecting data and reasoning across multiple document chunks is needed.

Amazon Bedrock Knowledge Bases automatically manages the creation and maintenance of the graphs from Amazon Neptune, so you can provide relevant responses to your end users, without relying on expertise in graph techniques.

Amazon Bedrock Knowledge Bases with GraphRAG offers the following benefits:

- More relevant responses by using contextual information from related entities and document sections.

- Better summarization by incorporating key content from your data sources while filtering out unnecessary information.

- More explainable responses by understanding the relationships between different entities in the dataset and providing citations.

**Note**: GraphRAG is available in AWS Regions where both Amazon Bedrock Knowledge Bases and Amazon Neptune Analytics are both available.

- For this module, we will use the Anthropic Claude 3 Haiku model as our FM to work with the max no. of results and prompt customization features

## Pre-requisites
Before being able to answer the questions, the documents must be processed and stored in a knowledge base. For this notebook, we use a `synthetic dataset for 10K financial reports` to create the Amazon Bedrock Knowledge Bases. 

1. Upload your documents (data source) to Amazon S3 bucket.

## Setup

In [ ]:
%pip install --upgrade pip --quiet
%pip install -r ../requirements.txt --no-deps --quiet
%pip install -r ../requirements.txt --upgrade --quiet

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

### Initialize boto3 client
Through out the notebook, we are going to utilise RetrieveAndGenerate to test knowledge base features.

In [ ]:
import json
import boto3
import pprint
import sys
from botocore.exceptions import ClientError
from botocore.client import Config
import time
import os
import logging

# Set the path to import module
from pathlib import Path
current_path = Path().resolve()
current_path = current_path.parent
if str(current_path) not in sys.path:
    sys.path.append(str(current_path))
    
from utils.knowledge_base import BedrockKnowledgeBase

# Create boto3 session
sts_client = boto3.client('sts')
boto3_session = boto3.session.Session()
region_name = boto3_session.region_name
account_id = sts_client.get_caller_identity()['Account']

# Create s3 client
s3_client = boto3.client('s3')

# Create bedrock agent client
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0}, region_name=region_name)
bedrock_agent_client = boto3_session.client("bedrock-agent-runtime",
                              config=bedrock_config)

# Define FM to be used for generations 
model_id = "anthropic.claude-3-haiku-20240307-v1:0" # we will be using Anthropic Claude 3 Haiku throughout the notebook
model_arn = f'arn:aws:bedrock:{region_name}::foundation-model/{model_id}'


In [ ]:
import time

# Get the current timestamp
current_time = time.time()

# Format the timestamp as a string
timestamp_str = time.strftime("%Y%m%d%H%M%S", time.localtime(current_time))[-7:]
# Create the suffix using the timestamp
suffix = f"{timestamp_str}"

knowledge_base_name = f"bedrock-sample-knowledge-base-{suffix}"
knowledge_base_description = "Graph RAG knowledge base."

bucket_name = f'{knowledge_base_name}-{account_id}'

## Create KB with Amazon Neptune as vector
To build GraphRAG, you must choose Amazon Neptune Analytics as your vector store.
The knowledge base automatically generates and stores document embeddings in Amazon Neptune, along with a graph representation of entities and their relationships derived from the document corpus.

When your GraphRAG-based application is running, you can continue using the Knowledge Bases API operations to provide end users with more comprehensive, relevant, and explainable responses.


In [ ]:
data_sources = [{"type": "S3", "bucket_name": bucket_name}]

In [ ]:
knowledge_base = BedrockKnowledgeBase(
    kb_name=f'{knowledge_base_name}',
    kb_description=knowledge_base_description,
    data_sources=data_sources,
    chunking_strategy = "GRAPH", 
    suffix = f'{suffix}-f',
    vector_store="NEPTUNE_ANALYTICS" # can be OPENSEARCH_SERVERLESS or NEPTUNE_ANALYTICS
)

## Download data to ingest into our knowledge base.
We'll use the following data:

sythetic data stored in a local directory as first data source
### Upload data to S3 Bucket data source

In [ ]:
import os
s3_client = boto3.client('s3')

def upload_directory(path, bucket_name):
        for root,dirs,files in os.walk(path):
            for file in files:
                file_to_upload = os.path.join(root,file)
                print(f"uploading file {file_to_upload} to {bucket_name}")
                s3_client.upload_file(file_to_upload,bucket_name,file)

upload_directory("../synthetic_dataset", bucket_name)

## Start ingestion job
Once the KB and data source(s) created, we can start the ingestion job for each data source. During the ingestion job, KB will fetch the documents in the data source, pre-process it to extract text, chunk it based on the chunking size provided, create embeddings of each chunk and then write it to the vector database, in this case Neptune Analytics.

NOTE: Currently, you can only kick-off one ingestion job at one time.

In [ ]:
# ensure that the kb is available
time.sleep(60)
# sync knowledge base
knowledge_base.start_ingestion_job()

In [ ]:
# keep the kb_id for invocation later in the invoke request
kb_id = knowledge_base.get_knowledge_base_id()
%store kb_id

## 2.2 Test the Knowledge Graph created using Amazon Bedrock Knowledge Bases
You can use the same `retreive` or `retrieve_and_generate` API based on your use case to query the `Neptune Analytics` graph created using Amazon Bedrock Knowledge Bases.

### Testing Knowledge Base with Retrieve and Generate API
We will first test it with `retrieve_and_generate` API which will now query the `Neptune Analytics` graph created using Amazon Bedrock Knowledge Bases to fetch the search results, augment it with the prompt and then use the provided foundation model to generate the response. 

`query = Provide a summary of consolidated statements of cash flows of Octank Financial for the fiscal years ended December 31, 2019.`

The right response for this query as per ground truth QA pair is:

```
According to the consolidated statements of cash flows, in the fiscal year ended December 31, 2019, Octank Financial had:
- Net cash provided by operating activities of $710 million
- Net cash used in investing activities of $240 million
- Net cash provided by financing activities of $350 million
- An overall net increase in cash and cash equivalents of $120 million
```

**NOTE**: Based on the model selected for generating responses, actual response may differ, however, will be semantically similar. 

In [ ]:
query = "Provide a summary of consolidated statements of cash flows of Octank Financial for the fiscal years ended December 31, 2019?"


In [ ]:
# foundation_model = "amazon.nova-micro-v1:0"

response = bedrock_agent_client.retrieve_and_generate(
    input={
        "text": query
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": model_arn,
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                } 
            }
        }
    }
)

print(response['output']['text'],end='\n'*2)


As you can see, with the retrieve and generate API we get the final response directly and we don't see the different sources used to generate this response. Let's now retrieve the source information from the knowledge base with the retrieve API.

### Testing Knowledge Base with Retrieve API
If you need an extra layer of control, you can retrieve the chunks that best match your query using the retrieve API. Since, the vector store is `Neptune Analytics`, the retrieve API will query the graph to fetch the relevant chunks (search results). In this setup, we can configure the desired number of results and control the final answer with your own application logic. The API then provides you with the matching content, its S3 location, the similarity score and the chunk metadata.

In [ ]:
response_ret = bedrock_agent_client.retrieve(
    knowledgeBaseId=kb_id, 
    nextToken='string',
    retrievalConfiguration={
        "vectorSearchConfiguration": {
            "numberOfResults":5,
        } 
    },
    retrievalQuery={
        "text": "How many new positions were opened across Amazon's fulfillment and delivery network?"
    }
)

def response_print(retrieve_resp):
#structure 'retrievalResults': list of contents. Each list has content, location, score, metadata
    for num,chunk in enumerate(response_ret['retrievalResults'],1):
        print(f'Chunk {num}: ',chunk['content']['text'],end='\n'*2)
        print(f'Chunk {num} Location: ',chunk['location'],end='\n'*2)
        print(f'Chunk {num} Score: ',chunk['score'],end='\n'*2)
        print(f'Chunk {num} Metadata: ',chunk['metadata'],end='\n'*2)

response_print(response_ret)


## Clean up
Please make sure to uncomment and run the below section to delete all the resources.

In [ ]:
# delete role and policies
print("===============================Deleting Knowledge Base and associated resources==============================\n")
knowledge_base.delete_kb(delete_s3_bucket=True, delete_iam_roles_and_policies=True)

<div class="alert alert-block alert-warning">
<b>Note:</b> Remember to delete KB, OSS index and related IAM roles and policies to avoid incurring any charges.
</div>